In [7]:
import pandas as pd
import re

In [9]:
filename = 'october2020.csv'
df = pd.read_csv(filename)
df.drop(df.columns[0], axis=1, inplace=True)
df.drop_duplicates('link', inplace=True)
df.set_index('link', inplace=True)
df.head()

,author,date,post,rawtext,title
link,,,,,
/r/mechmarket/comments/jaml20/usnyhgmk_olivia_dark_gmk_dualshot_w_gmk_handarbeit/,Phailure_,1.602624e+09,[timestamp](https://imgur.com/gallery/RuUqL8t)...,sold gmk olivia++ dark base to u/wavyhundreds,"[US-NY][H]GMK Olivia++ Dark, GMK DualShot [W] ..."
/r/mechmarket/comments/jhdf5l/usnyhnorbaforce_x2_lubrigante_olivia_dark/,Phailure_,1.603562e+09,[timestamp](https://imgur.com/gallery/mXImAjs)...,sold gmk olivia++ dark base to u/wavyhundreds,"[US-NY][H]Norbaforce x2, Lubrigante, Olivia++ ..."
/r/mechmarket/comments/jff2e3/usnyhlubrigante_norbaforce_mk1_kbd8x_pcb_olivia/,Phailure_,1.603294e+09,[timestamp](https://imgur.com/gallery/6iG806B)...,sold gmk olivia++ dark base to u/wavyhundreds,"[US-NY][H]Lubrigante, Norbaforce MK1, kbd8x pc..."
/r/mechmarket/comments/j0vrt9/usny_h_sealed_gmk_olivia_light_base_hihihi_rama/,SleepyMarket,1.601230e+09,[Time Stamp](https://imgur.com/a/By1Mvui)\r\n\...,sold gmk olivia++ light base and enter rama to...,[US-NY] [H] Sealed Gmk Olivia++ light base & H...
/r/mechmarket/comments/j1bv66/usca_h_gmk_olivia_light_base_w_paypal/,shunfu,1.601299e+09,~~BNIB GMK Olivia++ Light Base. $320 shipped C...,sold gmk olivia++ light to /u/so-amusing,[US-CA] [H] GMK Olivia++ Light Base [W] PayPal


In [10]:
df['post_lower'] = df['post'].str.lower()
df['title_lower'] = df['title'].str.lower()
only_gmk = df[df['post_lower'].str.contains('gmk') | df['title_lower'].str.contains('gmk')].copy()
not_gmk = df[(~(df['post_lower'].str.contains('gmk') | df['title_lower'].str.contains('gmk')))].copy()

only_gmk.shape, df.shape

((445, 7), (648, 7))

In [11]:
not_gmk.head()

,author,date,post,rawtext,title,post_lower,title_lower
link,,,,,,,
/r/mechmarket/comments/jff2e3/usnyhlubrigante_norbaforce_mk1_kbd8x_pcb_olivia/,Phailure_,1.603294e+09,[timestamp](https://imgur.com/gallery/6iG806B)...,sold gmk olivia++ dark base to u/wavyhundreds,"[US-NY][H]Lubrigante, Norbaforce MK1, kbd8x pc...",[timestamp](https://imgur.com/gallery/6ig806b)...,"[us-ny][h]lubrigante, norbaforce mk1, kbd8x pc..."
/r/mechmarket/comments/ix1cex/gb_key_switch_hook_earrings/,enby-girl,1.600698e+09,Hey folks! I posted [an interest check](https:...,bought gmk olivia++ light base and 41 kiwi swi...,[GB] Key Switch Hook Earrings,hey folks! i posted [an interest check](https:...,[gb] key switch hook earrings
/r/mechmarket/comments/j22mil/gb_reminder_and_giveaway_winner_key_switch_hook/,enby-girl,1.601397e+09,Hey folks! I thought I'd bump my GB for the Ke...,bought gmk olivia++ light base and 41 kiwi swi...,[GB] Reminder and giveaway winner: Key Switch ...,hey folks! i thought i'd bump my gb for the ke...,[gb] reminder and giveaway winner: key switch ...
/r/mechmarket/comments/iut48b/caqc_h_paypal_w_percent_volt_80_snow_white/,enby-girl,1.600379e+09,WTB: Percent Volt 80 Snow White.\r\n\r\nPrefer...,bought gmk olivia++ light base and 41 kiwi swi...,[CA-QC] [H] PayPal [W] Percent Volt 80 Snow White,wtb: percent volt 80 snow white.\r\n\r\nprefer...,[ca-qc] [h] paypal [w] percent volt 80 snow white
/r/mechmarket/comments/iuj8dj/ic_key_switch_hook_earrings/,enby-girl,1.600349e+09,I posted a pair of my custom/DIY mechanical sw...,bought gmk olivia++ light base and 41 kiwi swi...,[IC] Key Switch Hook Earrings,i posted a pair of my custom/diy mechanical sw...,[ic] key switch hook earrings


In [49]:
from collections import defaultdict
all_prices = defaultdict(list)

money_regex = r'\$(\d+)|(\d+)\$'

sets = ['base', 'nov', 'alpha', 'accent', 'bar', 'light base', 'dark base', 'deskmat', 'rama', '40s', '40\'s', 'fourties', 'mods', 'extension']

sales_data = []

def match_product(df):
    for row in df.itertuples():
        s = row.post_lower.split("\n")
        #product = row.product_name.lower()
        #print(row[0][1])
        for l in s:
            low = l.lower()
            if "gmk " in low and ('~~' in low or 'sold' in low):
                a = low.split("gmk ")[1]
                product_name = "gmk " + re.split(r"[^\w\+]", a)[0]
                matches = re.split(money_regex, low)
                temp_data = {}
                if len(matches) > 1:
                    #print(f"FULL LINE: {low}")
                    hadBase = False
                    allKits = []
                    for i in range(0, len(matches)-1, 3):
                        curr_price = int(matches[i+1] if matches[i+1] else matches[i+2])
                        curr_str = matches[i]
                        kits = []
                        removeBase = False
                        empty=True
                        for x in sets:
                            if x in curr_str:
                                empty=False
                                if x == 'nov':
                                    kits.append('novelties')
                                elif x == 'light base' or x == 'dark base':
                                    kits.insert(0, f'{x}')
                                    removeBase = True
                                elif x == 'bar':
                                    kits.append('spacebars')
                                elif x in ['40s', '40\'s', 'fourties']:
                                    kits.append('40s')
                                else:
                                    kits.append(x)
                        products = product_name
                        if i == 0 and not kits:
                            kits = ['base']
                        for x in kits:
                            if removeBase and x == 'base':
                                continue
                            products += f', {x}'
                        if i == 0:
                            temp_data["products"] = products
                            temp_data["str"] = curr_str
                            temp_data["price"] = curr_price
                        
                        if kits and i > 0:
                            #print(f"PRODUCT: {temp_data['products']} CURRENT STRING: {temp_data['str']} PRICE: {temp_data['price']}")
                            #all_prices[temp_data['products']].append(temp_data['price'])
                            sales_data.append([row[0], temp_data['products'], temp_data['price'], row.date])
                            if temp_data['products'] == 'gmk dark|base, base, novelties':
                                #print(row)
                                pass
                            temp_data["products"] = products
                            temp_data["str"] = curr_str
                            temp_data["price"] = curr_price
                        else:
                            temp_data["price"] = min(temp_data["price"], curr_price)
                    #print(type(temp_data['products']))
                    quit = False
                    if 'base' not in temp_data['products'] and 'stabs' not in low:
                        temp = low.split('gmk')[1]
                        for x in temp.split(' '):
                            for y in sets[1:]:
                                if y in x:
                                    quit = True
                            if quit: break
                                    
                            if '+' in x or ',' in x or '-' in x:

                                if x == 'olivia++':
                                    break
                                print(low)
                                print(row[0])
                                print(x)
                                a = temp_data['products']
                                temp_data['products'] = a[:a.index(',')] + ', base' + a[a.index(','):]

                                break
                            



                    #print(f"PRODUCT: {temp_data['products']} CURRENT STRING: {temp_data['str']} PRICE: {temp_data['price']}")
                    #all_prices[temp_data['products']].append(temp_data['price'])
                    sales_data.append([row[0], temp_data['products'], temp_data['price'], row.date])
                

for i, new_df in only_gmk.groupby(level=0):
    match_product(new_df)
    
#print(sales_data)

| ~~gmk red honey~~ | used for a few months, commonly used alphas and mod keys have some shine. comes with tray but can ship in bags if you really want.  don't have the banderole anymore but it was the generic gmk so nothing special. | $147 | n |
/r/mechmarket/comments/j0a020/usnyhpc_theo60_hotswap_gmk_red_honey_lubed_filmed/
months,
|[*~~^(gmk ceresia - red accent kit)~~*](https://imgur.com/gwqesbm)|*~~^(used - like new)~~*|*~~^(mounted for pictures only; no shine)~~*|*~~^($160 shipped conus)~~*|***sold!***|
/r/mechmarket/comments/j0w5v4/usmd_h_gmk_olive_gmk_plum_gmk_violet_on_cream_gmk/
-
gmk coral - brand new in box with spacebars. 280$ + shipping for the set and spacebars. ~~sold for asking~~
/r/mechmarket/comments/j0yxks/usca_h_ultramarine_fjell_gmk_coral_gmk_crimson/
-
gmk crimson cadet - mounted once for a few days. lightly used with no shine. spacebars are sealed. spacebars are in stock on novelkeys. 250$ + shipping for the set and space kit. ~~sold for asking~~
/r/mechmarket/c

In [45]:
sales_df = pd.DataFrame(sales_data, columns=['link', 'product', 'price', 'date'])
sales_df['date'] = pd.to_datetime(sales_df['date'], unit = 's')
sales_df.to_csv('results_2' + filename)